In [20]:
%pip install -U transformers
%pip install -U datasets
%pip install pytorch
%pip install torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-ug2gwmg6/pytorch_f83ec81e40e0457388f92cb828bf2b52/setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [21]:
%pip install accelerate>=0.21.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [22]:
from datasets import load_dataset

# Load the dataset
cnn_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:3%]")
cnn_dataset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 8613
})

In [23]:
cnn_dataset=cnn_dataset.train_test_split(test_size=0.3)

cnn_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 6029
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 2584
    })
})

In [24]:
def clean_text(example):
  for t in ["article"]:
    example[t]=example[t].lower()
    example[t]=example[t].replace("\\", "")
    example[t]=example[t].replace("/", "")
    example[t]=example[t].replace("\n", "")
    example[t]=example[t].replace("``", "")
    example[t]=example[t].replace('"', '')
    example[t]=example[t].replace("--", "")
  for t in ["highlights"]:
    example[t]=example[t].lower()
    example[t]=example[t].replace("\\", "")
    example[t]=example[t].replace("/", "")
    example[t]=example[t].replace("\n", "")
    example[t]=example[t].replace("``", "")
    example[t]=example[t].replace('"', '')
    example[t]=example[t].replace("--", "")


    return example

In [25]:
cleaned_cnn_dataset=cnn_dataset.map(clean_text)
cleaned_cnn_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 6029
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 2584
    })
})

In [26]:
cleaned_cnn_dataset['train']['article'][7]

"london, england (cnn)  the sole survivor of the crash that killed princess diana has told a court he still cannot remember the incident but does not support the conspiracy theories surrounding it. bodyguard trevor rees and the back of princess diana's head are seen shortly before the car crash. bodyguard trevor rees, formerly known as trevor rees-jones, was the front-seat passenger in the mercedes that carried diana, her boyfriend, dodi fayed, and their driver, henri paul. he sustained serious injuries in the august 31, 1997 crash and testified that he received anonymous phone calls and letters after the accident, threatening him to keep quiet. he said the caller told him to keep quiet, saying, we know who you are, we know where you are, and we know where you live. rees said he didn't take the calls or letters seriously. a lawyer also asked rees about a supposed encounter with a woman in which he told her, if i remember, they're going to kill me. rees said he didn't recall the convers

In [27]:
EXAMPLE_TEXT_INDEX= 1

example_text= cleaned_cnn_dataset['test']['article'][EXAMPLE_TEXT_INDEX]

example_text

"it's a disease that affects millions of women in africa, yet talking about it has been considered a taboo, and that silence has been deadly. nigeria's princess nikky onyeri has dedicated her life to speaking out to raise awareness about breast cancer in africa. princess nikky onyeri is not a member of a royal family, princess is a given name, but for many of nigeria's cancer survivors she is their princess. it's a passionate cause that defines her life. based in abuja, nigeria, onyeri has lobbied the government to do more to help women with cancer. the impetus behind her drive and persistence is a wrong diagnosis of cancer 15 years ago."

In [28]:
MODEL_NAME="t5-small"

In [29]:
from transformers import pipeline

prefix="summarize: "

summarizer=pipeline("summarization", model=MODEL_NAME, truncation=True)

summary_text=summarizer(prefix+example_text)

summary_text

Your max_length is set to 200, but your input_length is only 171. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=85)


[{'summary_text': "nigeria's princess nikky onyeri has dedicated her life to raising awareness about breast cancer in africa . the impetus behind her drive and persistence is a wrong diagnosis of cancer 15 years ago ."}]

In [30]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)

In [31]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model= AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [32]:
import torch
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

prefix="summarize: "

def preprocess_function(examples):

  inputs= [prefix + doc for doc in examples["article"]]
  model_inputs= tokenizer(inputs, max_length=1024, truncation=True)
  labels=tokenizer(text_target=examples["highlights"], max_length=128, truncation=True)
  model_inputs["labels"]= labels["input_ids"]

  return model_inputs

In [33]:
tokenized_cnn_dataset=cleaned_cnn_dataset.map(preprocess_function,batched=True)

tokenized_cnn_dataset

Map:   0%|          | 0/6029 [00:00<?, ? examples/s]

Map: 100%|██████████| 2584/2584 [00:01<00:00, 1616.13 examples/s]


DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6029
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2584
    })
})

In [34]:
from transformers import DataCollatorForSeq2Seq

data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=MODEL_NAME)

In [35]:
import accelerate
training_args=Seq2SeqTrainingArguments(
    output_dir="cnn_news_summary_model_trained_on_reduced_data",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size= 8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate= True,
    fp16=True,
)

In [36]:
trainer=Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_cnn_dataset['train'],
    eval_dataset=tokenized_cnn_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/home/ronitkirar/backend/en/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [37]:
PYTORCH_CUDA_ALLOC_CONF=expandable_segments=True

In [38]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.947700,1.615718
2,1.817100,1.598989
3,1.808400,1.594906


TrainOutput(global_step=2262, training_loss=1.8469199695806393, metrics={'train_runtime': 9993.8973, 'train_samples_per_second': 1.81, 'train_steps_per_second': 0.226, 'total_flos': 4888996039262208.0, 'train_loss': 1.8469199695806393, 'epoch': 3.0})

In [39]:
# import torch.cuda.amp as autocast
# input="The constitution is the cornerstone of a nation's governance, embodying its core values, principles, and aspirations. It serves as a solemn pact between the state and its citizens, delineating their respective powers, duties, and rights. Crafted with foresight and wisdom, a constitution lays down the framework for a just and equitable society, ensuring the rule of law and safeguarding individual liberties. At its essence, a constitution establishes the organs of government, defining their functions, structures, and interrelations. It establishes the separation of powers among the executive, legislative, and judicial branches, fostering a system of checks and balances to prevent the concentration of authority and the abuse of power. Furthermore, a constitution enshrines fundamental rights and freedoms, guaranteeing citizens protection against arbitrary actions by the state and promoting equality and justice for all. It serves as a bulwark against tyranny, providing avenues for recourse and redress when these rights are violated. In addition, a constitution often reflects the historical, cultural, and social fabric of a nation, incorporating provisions for diversity, pluralism, and inclusivity. It accommodates mechanisms for democratic participation, ensuring that the voices of all citizens are heard and valued in the governance process. During times of crisis or uncertainty, a constitution serves as a beacon of stability and resilience, guiding the nation through turbulent waters and upholding its democratic ideals. It is not a static document but a living instrument, capable of adaptation and evolution to meet the evolving needs and challenges of society. In essence, a constitution embodies the aspirations of a nation, shaping its identity and providing the framework for its progress and prosperity. It stands as a testament to the collective will of the people, forging a path towards a more perfect union grounded in justice, liberty, and dignity for all."
# tokenized_input=tokenizer(input,max_length=10000,truncation=True,return_tensors="pt")
# inputs=[prefix+input]
# model_inputs=tokenizer(inputs,max_length=10000,truncation=True,return_tensors="pt")
# model_inputs["input_ids"]=tokenized_input["input_ids"]
# with autocast():
#     summaries=model.generate(**model_inputs)
# generated_summary=tokenizer.decode(summaries[0],skip_special_tokens=True)
# print (generated_summary)
trainer.evaluate()

{'eval_loss': 1.5949060916900635,
 'eval_runtime': 358.9898,
 'eval_samples_per_second': 7.198,
 'eval_steps_per_second': 0.9,
 'epoch': 3.0}

In [40]:
# input="The constitution is the cornerstone of a nation's governance, embodying its core values, principles, and aspirations. It serves as a solemn pact between the state and its citizens, delineating their respective powers, duties, and rights. Crafted with foresight and wisdom, a constitution lays down the framework for a just and equitable society, ensuring the rule of law and safeguarding individual liberties. At its essence, a constitution establishes the organs of government, defining their functions, structures, and interrelations. It establishes the separation of powers among the executive, legislative, and judicial branches, fostering a system of checks and balances to prevent the concentration of authority and the abuse of power. Furthermore, a constitution enshrines fundamental rights and freedoms, guaranteeing citizens protection against arbitrary actions by the state and promoting equality and justice for all. It serves as a bulwark against tyranny, providing avenues for recourse and redress when these rights are violated. In addition, a constitution often reflects the historical, cultural, and social fabric of a nation, incorporating provisions for diversity, pluralism, and inclusivity. It accommodates mechanisms for democratic participation, ensuring that the voices of all citizens are heard and valued in the governance process. During times of crisis or uncertainty, a constitution serves as a beacon of stability and resilience, guiding the nation through turbulent waters and upholding its democratic ideals. It is not a static document but a living instrument, capable of adaptation and evolution to meet the evolving needs and challenges of society. In essence, a constitution embodies the aspirations of a nation, shaping its identity and providing the framework for its progress and prosperity. It stands as a testament to the collective will of the people, forging a path towards a more perfect union grounded in justice, liberty, and dignity for all."
# encoding=tokenizer(input,return_tensors="pt")
# encoding

In [41]:
trainer.save_model("my_model")

In [59]:

# Assuming you have a tokenizer and a model
from transformers import pipeline


# Step 1: Preprocess the input
text="Some of the best street food i've ever eaten was in asia. Every one of those little markets was filled with smells of delicious food that all centered around an asian staple: noodles. Learn to cook those properly and you'll be surprised how quick and easy it is to knock up a delicious dinner. First off, we're going to soak our noodles- now these are rice noodles- into a bowl and just pour the hot water over the rice noodles. Soak and rehydrate 12 to 15 minutes to soften up. If you've got a wok, perfect, if not just a normal size frying pan with big sloping sides. Pan on, get it really nice and hot. Chicken breast, very lean, hardly any fat on there. You've got this little fillet to slice that off. Now the secret to getting it really nice and thin and to cut it into strips. Take your knife, keep it nice and flat on the board and sort of slice it in half like a sort of scallop, like that we call this butterfly in the chicken. Take a rolling pin and nice and gently roll over the chicken. What it does? It sort of flattens. It allows you to slice it even thinner. And the thinner the slices of the chicken, the sort of crisper it gets, the quicker it cooks, slice it in half. Start slicing these nice thin slices, and the good news is, one chicken breast can serve two or three easily. Next, wash your knife and finely slice garlic. The thinner, the garlic the crispier. [Music]. This is a young, tender broccoli. Just slice them down. I want a bite to the broccoli, and normally you sort of paste the dish with one-third noodles, one-third vegetables and one-third of your protein chicken. [Music]. With a dish that takes literally minutes to put together, it's really important to get everything organized. Everything needs to be at your fingertips. Touch of olive oil, get a pan really nice and ready, just starting to smoke. Drop the chicken in first. [Music]. Salt pepper. Open up those little strands of chicken. Nice, once you've started to sear off the chicken. Get the garlic in now. Let that get really nice and crispy. The way to do it is to sort of spread all the chicken, the garlic, up the side of the pan. Can see why it's so important to cut the chicken thin strips: because it colors and cooks at the same time as well. Really, really crucial. Now that garlic's getting really nice and crispy, broccoli in feels strange putting raw broccoli in like that. Normally we'll blanch it in boiling water, dry it out. But you want that crunch now. Soy sauce that helps to season it but also stains beautifully. Soy sauce in lovely. It's exactly what i want now. Take that out. Give you a pan a little, wipe out a little teaspoon of olive oil. Get that wok really nicely oiled again. Drain your noodles. They're beautiful. This is a really exciting way of finishing this. Quick stir fry. Very classic: two eggs in [Music]. Whisk up the eggs and give that a really good whisk. [Music]. Sort of almost spread it at the side of the pan. Lightly season the eggs, noodles in, chicken and broccoli in and give that a really nice mix. You want the egg to sort of almost bring the dish together. That egg's cooked. [Music]. Make sure you've got that nice even distribution of chicken, broccoli, garlic, lovely, and then just finish that fresh lime and then you have a very simple, delicious stir-fry with rice noodles. [Music]. Making your enoki is so simple to do, yet the results are absolutely stunning and it's a great way of using up leftover baked potatoes. You can make noki just with flour and eggs. However, the potato gives it a nice light, sort of creamy, fluffy texture. Cut them in half, take your spoon and scoop the inside of those potatoes. I'm using leftover baked potatoes, but this really works as well with leftover ball potatoes. Two choices: you can get a fork and sort of mash the potato and get it nice and light and fluffy. Or this little gadget is called a ricer- i suppose it's a posh word for a potato masher- just squeeze gently, you can see how nice and light it is, almost like fluffy little strands of potato. You can do this when the potatoes are hot. It'll go through the rice so much quicker. Just slice that off there. [Music]. Now a nice spoon of okata ring, a little touch of salt and pepper. It's really important to season the mixture as we go along, otherwise the nokia becomes really bland flour over the ricotta sieve. So there's no lumps. One delicious egg. Give that a little whisk. Now make a little. Well, in the center. You want a nice, soft, pliable ball of dough. Give that a really good mix. Get some thyme flowers in there, and this time is light, fragrant and it's just a really nice herb and with the ricotta it tastes brilliant. Just pick the little tips, the time flowers. Next flour your hands generously and knead the mixture into a dough, fold in and push and basically what it's doing is get it nice and smooth as it starts to get a little bit wet and just add a little touch of flour. We want something really nice and soft. Now, don't overwork it, it stops the nokia from expanding when it hits the pan. That's exactly what i want: a nice sort of soft, fragrant ball. Cut the ball in half, lightly flour the hands and just roll it gently. Just think of a big, long cigar. The mixture will start getting a little bit sort of wetter. But do not add lots of flour now. Lightly flour the knife so when you slice the gnocchi it doesn't stick. Cut the dough into bite-sized pieces, just take your finger, dip it in the flour and push down. Why i want my knocking to look like a pillow? And for me the most important part there is that not one of them are identically the same shape. Water on, bring up to the boil. Little touch of olive oil in there, lightly flour your hand. Lift up the nokia in to the rolling boiling water. Turn that pan to stop them from sticking at the bottom and let them simmer and they start to sort of tell you they're cooked when they start floating. Get a pan on. Get that nice and hot. Now they're just starting to come up to the top, and you can continue cooking them like that. I like blanching them in the water, taking them out and then frying them to study the gnocchi. Heat olive oil in a frying pan, gently lift up and look, they've doubled in size. Drain it, get rid of the excess water and straight in to the hot pan. [Music]. This is where they start to take on a completely different texture: nice, crisp, sauteed texture on the outside. Nokia loves fresh pepper, so pepper in and you see, as i start turning them, got this really nice little sort of brown color and they're almost popping up now like little parcels. So i want them nice and sauteed both sides, but light and creamy in the center. Fresh garden peas in and the butter gives it a really nice sort of bernoulli flavour on the end. Beautiful. Put a little bit of fresh thyme over the peas and then finally- i want to lift it up- fresh lemon zest the lemon over [Music], so smells incredible. And then finally seal the deal with a touch of grated piles and cheese. Give your veg some attitude and you'll get amazingly elegant dishes on a budget that are always guaranteed to impress. What more do you want from great cooking? Cheap, to make, easy to cook and absolutely stunning. For me, food always has to be impressive, but when it comes to desserts, often you see spun sugar or wild decorations. Remember, simple is always the most impressive. This cheesecake is so straightforward, yet so delicious. Now cream cheese. Leave it out the fridge for five or ten minutes. Go nice and soft. Trust me, your arms will be thanking you. Sugar in this cheesecake is new york cheesecake. Because it's baked, it says no base. Start creaming the cheese and the sugar. Spending the amount of time i do in the states. If there's one thing they know how to do, out there is the most amazing, impressive cheesecake. Rich, delicious, but so simple. Work the bowl, lift the bowl to your advantage. Really. Whisk, whisk, whisk, whisk. Lovely, nice and creamy. Now get your eggs. Add the eggs to the mixture, bit by bit. Doing it this way, it's more efficient. Hey, you're incorporating a lot of air b. The mixture doesn't separate [Music] the last of your egg. Lovely, of course you can use electric mixer, but why go to the gym? We can just make a cheesecake a day. A cheesecake a day. Keep the bingo wings away. Now a couple of tablespoons of flour. Give it a whisk. Stop something going lumpy. Now i want to scent that cheesecake. I've got the freshness zest, the lemon, in there. I want to sort of tart it up even more. Fold in some fresh raspberries. So just mix them through, be careful to crush them. Then grease a cake tin with butter. This will ensure your cheesecake slides out beautifully. Get your mix, let that fall in. Now take your cake tin and just tap it. The mixture hits the bottom of the cake tin. The raspberries rise and you've got raspberries: the top, the middle and the bottom. It also stops all those little pockets of air trapping underneath the mixture in the cake tin. There's no holes in the cheesecake. Now into the oven, 180 degrees for 35 to 40 minutes. [Music]: slightly souffled up. Pull off- that's the color i wanted on top and look at it. It's one of the simplest yet the most stylish cheesecakes anywhere. Beautiful [Music] you."
text="summarize: "+text
a=[i for i in text.split()]
max_output_size=int(int(len(a)))
summarizer=pipeline("summarization",model="my_model",min_length=int(max_output_size/4), max_length=max_output_size)
pred=summarizer(text)
print(max_output_size)

preds=[pred[0]['summary_text']]

print (preds)


Token indices sequence length is longer than the specified maximum sequence length for this model (2353 > 512). Running this sequence through the model will result in indexing errors


1737
["new york cheesecake is one of the simplest yet the most stylish cheesecakes in the world .make sure you've got that nice, soft, pliable ball of dough .get some thyme flowers in there, and this time is light, fragrant and it's just a really nice herb and with the ricotta it tastes brilliant .you can make your enoki just with flour and eggs, but don't overwork it, it stops the nokia from expanding when it' 'home', 'the ... .........the most i ..it's a 'd.com a...... and ...and ... just ... and the . and . the 'no a little bit of . ... the sydd a sweet & .so ...the cheesecake, .... and then ..............the a lot of tyron ...seared a bit of the whole .the .as the i want a. .?..........and the noodle & cheese in a way .o.....and the cheesecake &... ad .ees ...yead the ay'd sads in . it......just ...just the ................................................. ......... ... it ...?...and just .!..how i’d if you’d love a,...... ?... .... ...it’s s........................................